In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from dotenv import load_dotenv
import os
from tqdm import tqdm
import zipfile
from pathlib import Path

In [ ]:
load_dotenv()
edinet_subscription_key = os.environ['EDINET_SUBSCRIPTION_KEY']

In [ ]:
save_path = Path('edinet_downloads')
save_path.mkdir(parents=True, exist_ok=True)

In [ ]:
# APIエンドポイント
url = "https://api.edinet-fsa.go.jp/api/v2/documents.json"
    
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

date_range = []
current_date = start_date
while current_date <= end_date:
    date_range.append(current_date)
    current_date += timedelta(days=1)

all_documents = []

for current_date in tqdm(date_range, desc='データ取得中', unit='日'):
    date_str = current_date.strftime("%Y-%m-%d")

    # リクエストパラメータ
    params = {
        "date": date_str,  # 今日の日付
        'type': 2,
        "Subscription-Key": edinet_subscription_key  # あなたのSubscription-Keyに置き換えてください
    }
    
    # GETリクエストを送信
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    # JSONレスポンスを取得
    data = response.json()
    all_documents.extend(data['results'])

In [ ]:
all_documents_df = pd.DataFrame(all_documents)
target_documents_df = all_documents_df[pd.notna(all_documents_df['secCode']) & (all_documents_df['docTypeCode'] == '120') & (all_documents_df['formCode'] == '030000')]
target_documents_df.to_excel('edinet_target_documents.xlsx')

In [ ]:
doc_id = 'S100UBDZ'

url = f'https://api.edinet-fsa.go.jp/api/v2/documents/{doc_id}'

params = {
    'type': 5,
    'Subscription-Key': edinet_subscription_key
}

response = requests.get(url, params=params)
response.raise_for_status()

zip_filename = save_path / f'{doc_id}.zip'
with open(zip_filename, 'wb') as f:
    f.write(response.content)

extract_dir = save_path / doc_id
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

csv_dir = extract_dir / 'XBRL_TO_CSV'
csv_files = list(csv_dir.glob('*.csv'))

df_list = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file, encoding='utf-16-le', sep='\t')
    df_list.append(df)
combined_df = pd.concat(df_list, axis=0, ignore_index=True)